This notebook contains the code for an interactive dashboard for making [Datashader](http://datashader.org) plots from any dataset that has latitude and longitude (geographic) values. Apart from Datashader itself, the code relies on other Python packages from the [HoloViz](http://holoviz.org) project that are each designed to make it simple to:

- lay out plots and widgets into an app or dashboard, in a notebook or for serving separately ([Panel](http://panel.pyviz.org))
- build interactive [Bokeh](http://bokeh.pydata.org)-based plots backed by Datashader, from concise declarations ([HoloViews](http://holoviews.org) and [hvPlot](http://hvplot.pyviz.org))
- describe the information needed to load and plot a dataset, in a text file ([Intake](http://intake.readthedocs.io))

This notebook provides the same functionality as the [Param-based Datashader dashboard](https://examples.pyviz.org/datashader_dashboard/dashboard.html), but this version defines explicit widgets rather than defining a class with Parameters. Defining explicit widgets can be simpler as long as your code is only intended for use with a GUI app, while the Param-based approach is less direct and obvious but can allow your code to be used with or without an associated GUI. You can compare the code in this notebook line by line with the Param-based version to see how the two approaches are similar and differ. This version requires HoloViews >= 1.12.6 and Panel >= 0.6.4.

In [ ]:
import os, colorcet, holoviews as hv, panel as pn, datashader as ds, intake
from holoviews.element import tiles as hvts
from holoviews.operation.datashader import rasterize, shade, spread
from collections import OrderedDict as odict

hv.extension('bokeh', logo=False)

This notebook depends on data assumed to be available at locations specified in the associated `catalog.yml` file. If you obtained this notebook by cloning https://github.com/pyviz-topics/examples, then you can fetch the data and run this notebook as a dashboard in a single step for any of the supported datasets:

```
anaconda-project run nyc_taxi
```

(where `nyc_taxi` can be replaced with any of the available datasets (`nyc_taxi`, `nyc_taxi_50k` (tiny version), `census`, or `osm-1b`). If you already have these data files, or if you have edited `catalog.yml` to describe and point to your own datasets, you can launch the dashboard server and open a new browser tab with a command like:

```
DS_DATASET=nyc_taxi panel serve --show dashboard.ipynb
```

To launch multiple dashboards at once, you'll need to add `-p 5001` (etc.) to select a unique port number for the web page to use for communicating with the Bokeh server.  Otherwise, be sure to kill the server process before launching another instance. If you just want to run the notebook in Jupyter, you can do so with `DS_DATASET=nyc_taxi jupyter notebook dashboard.ipynb`, again replacing the dataset with your preferred one or editing the default below.

In [ ]:
dataset = os.getenv("DS_DATASET", "nyc_taxi_50k")
catalog = intake.open_catalog('catalog.yml')
source  = getattr(catalog, dataset)

The Intake `source` object here lets us treat data in many different formats the same in the rest of the code below. We can now create some widgets for values that the user can vary:

In [ ]:
plots  = odict([(source.metadata['plots'][p].get('label',p),p) for p in source.plots])
fields = odict([(v.get('label',k),k) for k,v in source.metadata['fields'].items()])
aggfns = odict([(f.capitalize(),getattr(ds,f)) for f in ['count','sum','min','max','mean','var','std']])

norms  = odict(Histogram_Equalization='eq_hist', Linear='linear', Log='log', Cube_root='cbrt')
cmaps  = odict([(n,colorcet.palette[n]) for n in ['fire', 'bgy', 'bgyw', 'bmy', 'gray', 'kbc']])

maps   = ['EsriImagery', 'EsriUSATopo', 'EsriTerrain', 'CartoMidnight', 'StamenWatercolor', 'StamenTonerBackground']
bases  = odict([(name, hvts.tile_sources[name]().relabel(name)) for name in maps])
gopts  = hv.opts.Tiles(responsive=True, xaxis=None, yaxis=None, bgcolor='black', show_grid=False)

In [ ]:
plot          = pn.widgets.Select(name='Plot', options=plots)
field         = pn.widgets.Select(name='Field', options=fields)
agg_fn        = pn.widgets.Select(name='Agg_fn', options=aggfns)
    
normalization = pn.widgets.Select(name='Normalization', options=norms)
cmap          = pn.widgets.Select(name='Colormap', options=cmaps)
spreading     = pn.widgets.DiscreteSlider(name="Spreading", value=0, options=list(range(0,6)))
    
basemap       = pn.widgets.Select(name='Basemap', options=bases, value=bases['EsriImagery'])
data_opacity  = pn.widgets.FloatSlider(name="Data opacity", value=1.00, start=0, end=1)
map_opacity   = pn.widgets.FloatSlider(name="Map opacity", value=0.75, start=0, end=1)
show_labels   = pn.widgets.Checkbox(name="Show labels", value=True)

widgets = pn.Column(plot, field, agg_fn, normalization, cmap, spreading,
                    basemap, data_opacity, map_opacity, show_labels)

Each of these widgets includes an underlying value, available as e.g. `field.value`, and underlying "Parameter" object (a container for a value), available as e.g. `field.param.value`. You can see and work with the widgets by putting the word `widgets` at the end of a Jupyter notebook cell, but they won't do anything useful yet because they have not been connected to anything.

We can now specify code for updating a plot of the data source that will respond to changes in the parameter value as the widget is adjusted:

In [ ]:
@pn.depends(plot=plot)
def elem(plot):
    return getattr(source.plot, plot)()

@pn.depends(field=field, agg_fn=agg_fn)
def aggregator(field, agg_fn):
    field = None if field == "counts" else field
    return agg_fn(field)

@pn.depends(map_opacity=map_opacity, basemap=basemap)
def tiles(map_opacity, basemap):
    return basemap.opts(gopts).opts(alpha=map_opacity)

@pn.depends(show_labels = show_labels)
def labels(show_labels):
    return hvts.StamenLabels().opts(level='annotation', alpha=1 if show_labels else 0)

rasterized = rasterize(hv.DynamicMap(elem), aggregator=aggregator, width=800, height=400)
shaded     = shade(rasterized, cmap=cmap, normalization=normalization)
spreaded   = spread(shaded, px=spreading, how="add")
dataplot   = spreaded.apply.opts(alpha=data_opacity, show_legend=False)
viewable   = hv.DynamicMap(tiles) * dataplot * hv.DynamicMap(labels)

If we put `viewable` at the end of a Jupyter cell, we'll get a plot that displays itself in a notebook cell.  Moreover, because of how we declared the dependencies between each bit of code and each widget, the corresponding part of that plot will update whenever one of the widgets is changed on it. (Try putting `viewable` in one cell, then set some parameter like `spreading.value=4` in another cell.) But since what we want is the user to be able to manipulate the values using graphical widgets rather than Python code, let's go ahead and create a full dashboard out of this object by laying out a logo, the widgets, and the viewable object:

In [ ]:
logo = "https://raw.githubusercontent.com/pyviz/datashader/master/doc/_static/logo_horizontal_s.png"

panel = pn.Row(pn.Column(logo, widgets), viewable)
panel.servable("Datashader Dashboard")

If you are viewing this notebook with a live Python server process running, adjusting one of the widgets above should now automatically update the plot, re-running only the code needed to update that particular item without re-running Datashader if that's not needed. It should work the same when launched as a separate server process, but without the extra text and code visible as in this notebook. Here the `.servable()` method call indicates what should be served when run as a separate dashboard with a command like `panel serve --show dashboard.ipynb`, or you can just copy the code above out of this notebook into a `dashboard.py` file then do `panel serve --show dashboard.py`.

## How it works

You can use the code above as is, but if you want to adapt it to your own purposes, you can read on to see how it works. 


### Overview

The code has three main components:

1. Data: A dataset `source` with associated metadata managed by [Intake](http://intake.readthedocs.io), which allows this notebook to ignore details like:
   - File formats
   - File locations
   - Column and field names in the data<br><br>
   Basically, once the `source` has been defined in the cell starting with `dataset`, this code can treat all datasets the same, as long as their properties have been declared appropriately in the `catalog.yml` file. Intake objects support `.plot`, which uses [hvPlot](http://hvplot.pyviz.org) to return a HoloViews and Bokeh-based plot object that is used in the later steps below.<br><br>

2. Panel widgets and plotting functions, specifying:
   - What widgets we want the user to be able to manipulate
   - How to generate the overall plot specified by those widget values, starting from the basic [hvPlot](http://hvplot.pyviz.org)-based object and modifying it using [HoloViews](http://holoviews.org) and [Datashader](http://datashader.org).
   - Which functions need to be run when one of the widget values changes<br><br>
   
3. A Panel layout `panel`: A [Panel](http://panel.pyviz.org)-based app/dashboard consisting of:
   - a logo (just for pretty!)
   - The widgets defined above.
   - The `viewable` dynamic HoloViews object.

You can find out more about how to work with these objects at the websites linked for each one. 

If you want to start working with this code for your own purposes, parts 1 and 3 should be simple to get started with. You should be able to add new datasets easily to `catalog.yml` by copying the description of the simplest dataset (e.g. `osm-1b`). If you wish, you can then compare that dataset's description to the other datasets, to see how other fields and metadata can be added if you want there to be more options for users to explore a particular dataset. Similarly, you can easily add additional items to lay out in rows and columns in the `panel` app; it should be trivial to add anything Panel supports (text boxes, images, other separate plots, etc.) to this layout as described at [Panel.org](http://panel.pyviz.org). 

### Expressing user-defined values and dependencies

Part 2 (the widgets and plotting functions) is the hard part to specify, because that's where the complex relationships between the user-settable widget values and the underlying behavior are expressed. 

Before we try to understand the full code above, let's consider a simpler case with a single callback. What if our dataset is so small (e.g. `nyc_taxi_50k` with only 50,000 points) that it would be ok to update the plot every time any widget changed? In that case we could get away with a brute-force, one-function approach we'll call `view`:

In [ ]:
@pn.depends(agg_fn=agg_fn, basemap=basemap, cmap=cmap, data_opacity=data_opacity, field=field,
    map_opacity=map_opacity, normalization=normalization, plot=plot, show_labels=show_labels, spreading=spreading)
def view(agg_fn, basemap, cmap, data_opacity, field, map_opacity, normalization, plot, show_labels, spreading):
    base       = basemap.opts(gopts).opts(alpha=map_opacity)

    field      = None if field == "counts" else field
    rasterized = rasterize(hv.DynamicMap(getattr(source.plot, plot)), 
                           aggregator=agg_fn(field), width=800, height=400)
    shaded     = shade(rasterized, cmap=cmap, normalization=normalization)
    spreaded   = spread(shaded, px=spreading, how="add")
    dataplot   = spreaded.opts(alpha=data_opacity, show_legend=False)
        
    labels     = hvts.StamenLabels().opts(level='annotation', alpha=1 if show_labels else 0)
    return        dataplot * labels ### This line is temporary and should not be needed
    return base * dataplot * labels

This `view` function declares that it depends on all of the listed widget values (`agg_fn`, `basemap`, `cmap`, and so on). If `view` and the `widgets` are in a Panel, whenever one of the widget values changes, Panel will call the `view` function with the current widget values and construct a plot appropriately. In this case the resulting plot is a [HoloViews](http://holoviews.org) `Overlay` of three components: (1) the underlying map tiles `base` (like Google Maps), (2) the [datashaded](http://datashader.org) `dataplot` (using the aggregation (rasterization), colormapping (shading) with normalization, and spreading functionality from Datashader), and (3) overlaid geographic `labels` (which also happen to be a tile-based map, but with only text).  

We can use the decorated `view` function much as we used the `viewable` object above. I.e., if it's embedded in a Panel layout with its widgets, you'll see that everything works much the same as `viewable` does, with the plot updating appropriately when any of the widgets changes.

In [ ]:
pn.Row(widgets,view)

This approach will work for essentially _anything_ that can be returned by a custom function, including plots from nearly any supported library. So why is the real `viewable` object so much more complex above, being built from all those separate functions and separately enumerated dependency declarations? Well, if you do try the `view` version, you should be able to see that it works but it is not very responsive, because Panel re-runs the whole `view()` function every single time any widget changes value. Re-running everything makes it possible to support every library, but even for a 50k dataset, the resulting plot flickers any time a widget is used. For a larger dataset there can be a very annoying lag, as the entire plot is rebuilt from scratch. Slider widgets in particular can be very confusing with a lag like that, making it difficult to explore the data. So this simple version is not the most usable, but it's still a good first pass -- it makes all the right widgets and connects them all up to control the plot that you see. If that's good enough, then you can stop there!

### Expressing fine-grained dependencies

So what if we do anticipate working with larger files and still want the interface to be responsive wherever possible, using HoloViews? The full `viewable` object and associated functions show how to ensure that only the specific bits of code that depend directly on each parameter are re-run when that widget is changed, making it highly reponsive even with large datasets.  For instance, the `map_opacity` slider affects only the underlying map tiles, and so with `viewable` that slider can be dragged with instantaneous updating regardless of the dataset size; the data plot stays the same while the tiles update.  The `spreading` and `cmap` widgets do need to access the data, but even they can still be very fast, because they affect only the very last step in the data processing, after aggregation but before the final display.

So, how is this fine-grained control over bits of computation achieved? First, you'll notice that `viewable` is named as an adjective rather than the imperative "`view()`" function that is a verb. We provided the `view` function directly to Panel, and Panel keeps track of its dependencies and calls `view` every time a widget's parameter changes, generating a completely new plot.  But `viewable` is a fixed HoloViews object, not a function call, with this object provided to Panel.  `viewable` is a dynamically updatable object from HoloViews whose parts are precisely tied internally to each of the relevant parameters. (Hence the perhaps too-subtle difference in the names of those two items; one is a command to view immediately, and the other is a viewable object that can be kept around and viewed at will.) 

To understand the `viewable` object, first consider a simpler version that doesn't display the data at all:

```
viewable2 = hv.DynamicMap(tiles) * hv.DynamicMap(labels)
```

Here, `hv.DynamicMap(callback)` returns a dynamic HoloViews object that calls the provided `callback` whenever the object needs updating.  When given a Panel-decorated function, `hv.DynamicMap` understands any dependencies that have been declared for that function.  In this case, the map tiles will thus be dynamically updated whenever the `map_opacity` or `basemap` widgets change, and the overlaid labels will be updated whenever the `show_labels` widget changes (because those are the relationships expressed on `def tiles()` and `def labels()` with the `pn.depends` decorator in their declaration).  The `viewable2` object here is then an overlay (constructed by the `*` syntax for HoloViews objects), retaining the underlying dynamic behavior of the two overlaid items.

Still following along? If not, try viewing `viewable2` and playing around with the source code to see how those parts fit together. Once that all makes sense, then we could add in a plot of the actual data:

```
viewable3 = hv.DynamicMap(tiles) * hv.DynamicMap(elem) * hv.DynamicMap(labels)
```

Just as before, we use a `DynamicMap` to call the `.elem()` method whenever one of its parameter dependencies changes (`plot` in this case).  Don't actually try to display `viewable3`, though, unless you have a very small dataset (even the tiny `nyc_taxi_50k` may be too large for some browsers).  As written, this code will pass all the data on to your browser, with disastrous results for large datasets!  This is where Datashader comes in; to make it safe for large data, we can instead wrap this object in some HoloViews operations that turn it into something safe to display:

```
viewable4 = hv.DynamicMap(tiles) * spread(shade(rasterize(hv.DynamicMap(elem)))) * hv.DynamicMap(labels)
```

This version is now runnable, with `rasterize()` dynamically aggregating the data using Datashader whenever a new plot is needed, `shade()` then dynamically colormapping the data into an RGB image, and `spread()` dynamically spreading isolated pixels so that they become visible data points.  But if you try it, you'll notice that the plot is ignoring all of the rasterization, shading, and spreading parameters we declared above, because those parameters are not declared as dependencies of the `elem` method that was given to this DynamicMap.  

We could of course add those parameters as dependencies to `.elem`, but if we do that, then the whole set of chained operations will need to be re-run every time any one of those parameters changes, just like for the clunky `view()` function above. For a large dataset, re-running all those steps can take seconds or even minutes, yet some of the changes only affect the very last (and very cheap) stages of the computation, such as `spread` or `shade`. 

So, we come to the final version of `viewable` that's used in the actual code above, which creates a whole slew of chained `hv.DynamicMap` objects that each dynamically respond to _some_ of the possible user actions:
- `hv.DynamicMap(elem)` returns an appropriate HoloViews element whenever the `plot` widget changes
- `rasterized` applies the Datashader aggregation operation to the result of `hv.DynamicMap(elem)` whenever that result changes or when the dependencies of the `aggregator` function change (the `field` and `agg_fn` widgets)
- `shaded` applies the Datashader shading operation to the result of `rasterized` whenever that result changes or the `cmap` and `normalization` widgets change.
- `dataplot` sets options on the result of `shaded` whenever that result changes or the `data_opacity` widget value changes.

So far we have only discussed how `pn.depends()` allows a function to declare its dependencies, but there are actually currently three different ways to set up dynamic, responsive behavior with Panel widgets, of which `viewable` uses methods 2 and 3:

1. **Function dependency for Panel**: Decorating a function definition with  `pn.depends(name = widget)` (equivalent to `pn.depends(name = widget.param.value)`), and passing that function to Panel so that Panel will call the function when any of the dependencies changes (used for the clunky `view()` function, but not for the responsive `viewable` object). Completely general, but very coarse-grained; useful for a first pass, for simple cases, and for plotting libraries not supporting more fine-grained approaches.
2. **Function dependency for HoloViews DynamicMap**: Decorating a function with `pn.depends(name = widget)`, and passing that function to a HoloViews DynamicMap so that HoloViews will call the function when any of the dependencies change (used for most of the functions above).
3. **Widget instance-object argument**: Supplying a Panel widget object (or its associated Parameter object `obj.param.value`) as an argument to a HoloViews Operation, DynamicMap, or `.apply()` call, instead of supplying a concrete value like an integer or float. Providing the widget or its underlying parameter will cause that operation to re-run its computation when the widget value changes.  This approach is used for the `dataplot` object, which responds dynamically to `cmap`, `normalization`, `spreading`, and `data_opacity` because those widgets are supplied like  `px=spreading` (the widget) or `px=spreading.param.value` (the widget's Parameter object) rather than `px=spreading.value` (which would immediately look up the current value and would thus simply be equivalent to `px=0` if the current value of `spreading` is 0).  Dependencies are inferred only when the whole widget or its underlying Parameter object is supplied, not just the current value of the widget.

Approaches 2 and 3 are based on a feature of HoloViews called [streams](http://holoviews.org/user_guide/Responding_to_Events.html#Introducing-streams), which support many types of dynamic behavior other than responding to widgets. For instance, the `rasterize` operation attaches a `RangeXY` stream that re-aggregates the data whenever the viewport (x or y range) changes, as a user zooms or pans a Bokeh plot. Other streams can be created manually to perform custom behavior, such as consuming streaming data sources, reacting to arbitrary plot events, and so on.

These sources of dynamic behavior make the `dataplot` chain of HoloViews DynamicMaps be richly interactive. The interactive `dataplot` is then overlaid with `hv.DynamicMap(base)` (which itself is updated when the `map_opacity` and `basemap` widgets change), and with `hv.DynamicMap(labels)` (which itself is updated when the `show_labels` widget changes). Now, each part of the plot updates only if the relevant widgets have changed, and otherwise is left as it was, avoiding flicker and providing snappy performance.

As you can see, if we want to be very careful to tie each bit of computation to the values that affect it, then we can precisely determine which code to re-run interactively, providing maximal responsiveness where possible (try dragging the opacity sliders or selecting colormaps), while re-running everything when needed (when aggregation-related widgets change). In your own Panel dashboards and apps, you can often use the simplest approach (which can be as simple as a one-line call to [interact](https://panel.pyviz.org/user_guide/Interact.html)), but it is important to know that fine-grained control is available when you need it, and is still typically vastly more concise and explicitly expressed than with other dashboarding approaches.